In [2]:
import os
import re
import shutil
import pandas as pd
from tqdm import tqdm


In [ ]:

print("Iniciando cópia renomeada de cada arquivo Excel na pasta 'testes'...")

# Diretório contendo os arquivos a serem processados
diretorio = 'invalido'  # ajuste para o caminho real se for outro
# Pasta de destino para arquivos renomeados:
testes_dir = os.path.join('testes')
# Cria (ou limpa) a pasta testes
if os.path.exists(testes_dir):
    for f in os.listdir(testes_dir):
        os.remove(os.path.join(testes_dir, f))
else:
    os.makedirs(testes_dir)

# Lista de arquivos Excel originais
arquivos_origem = [os.path.join(diretorio, f) for f in os.listdir(diretorio)
                   if f.lower().endswith(('.xlsx', '.xls'))]

# Função para gerar nome com base no valor de C2 apenas
def gerar_novo_nome(arquivo):
    try:
        df = pd.read_excel(arquivo, header=None, dtype=str)
    except Exception as e:
        print(f"[ERRO] Falha ao ler {arquivo}: {e}")
        return None

    numero = str(df.iloc[1, 2]).strip()
    if not numero or numero.lower() == 'nan':
        print(f"[AVISO] Célula C2 vazia ou inválida em {arquivo}")
        return None

    nome_base, ext = os.path.splitext(os.path.basename(arquivo))
    novo_base = re.sub(r'\d+', numero, nome_base)
    return novo_base + ext

# Processa cada arquivo: gera novo nome e copia já renomeando
for arquivo in tqdm(arquivos_origem, desc="Processando arquivos", total=len(arquivos_origem)):
    nome_original = os.path.basename(arquivo)
    novo_nome = gerar_novo_nome(arquivo)
    if not novo_nome:
        continue

    destino = os.path.join(testes_dir, novo_nome)
    if os.path.exists(destino):
        print(f"[AVISO] '{novo_nome}' já existe em 'testes', pulando...")
        continue

    try:
        shutil.copy2(arquivo, destino)
        print(f"[OK] '{nome_original}' copiado e renomeado para '{novo_nome}'")
    except Exception as e:
        print(f"[ERRO] Falha ao copiar e renomear {nome_original}: {e}")

print("Processamento concluído.")


In [ ]:
# df_clientes = pd.read_csv("projects_data/Itau/OneDrive_2024-10-21/tela_preta/df_clientes.csv", sep=";")
# df_result_bens = pd.read_csv("projects_data/Itau/OneDrive_2024-10-21/tela_preta/df_result_bens.csv", sep=";")
# df_result_contrato = pd.read_csv("projects_data/Itau/OneDrive_2024-10-21/tela_preta/df_result_contrato.csv", sep=";")
# df_composicao_saldo_devedor = pd.read_csv("projects_data/Itau/OneDrive_2024-10-21/tela_preta/df_composicao_saldo_devedor.csv", sep=";")
# df_result_bem_l7ry = pd.read_csv("projects_data/Itau/OneDrive_2024-10-21/tela_preta/df_result_bem_l7ry.csv", sep=";")
# file_paths = ["projects_data/Itau/relatorios/dados_contrato_final_teste.xlsx",
#     "projects_data/Itau/relatorios/dados_contrato_lote2.xlsx",
#     "projects_data/Itau/relatorios/dados_contrato_lote3.xlsx"]
# comparativo = pd.read_excel("projects_data/Itau/relatorios/Quadro Comparativo de Informações por Origem e Status_docs_v7.xlsx",dtype={'Contrato':str})

path_base = "projects_data/Itau/DOC_BASE_Reprocessado (1).xlsx"


# Processar telas pretas

In [ ]:

def f4_Tela_Consulta_de_bens(texto):
    padrao = r"DT\.LIQUIDACAO:\s*(\d{2}/\d{2}/\d{2})"
        
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)

    dt_liquidacao = resultado.group(1) if resultado else None
    
    padrao = r"CONTRATO:\s*(\d+)"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna o número do contrato encontrado ou None
    n_contrato = resultado.group(1) if resultado else None
    
        # Expressão regular para encontrar "VL.UNITARIO: " seguido de um valor monetário
    padrao = r"VL\.UNITARIO:\s*([\d\.]+,\d{2})"
    
    # Procurar no texto por todas as ocorrências
    valores = re.findall(padrao, texto)
    # print(valores)
    # Converter os valores encontrados para float e somar
    valor_bem = sum(float(valor.replace('.', '').replace(',', '.')) for valor in valores)
    

    
    return [n_contrato, dt_liquidacao, valor_bem]


['5076788', '13/06/22', 2184000.0]

In [75]:
def Tela_Contrato_L7RR(texto):
    padrao = r"EMP/CONTRATO:\s*([\d/]+)"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna o número do contrato encontrado ou None
    n_contrato = resultado.group(1) if resultado else None
    
    padrao = r"CLIENTE:\s*([A-Z\s\.&]+?)(?=\s+(?:AG/CONTCGC|CPF|AG/CONTA-D/TIT|AG/CONTA-D/CPF))"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna o nome do cliente encontrado ou None
    cliente = resultado.group(1).strip() if resultado else None
    
    padrao = r"DT\.CONTRAT\.INICIAL:\s*(\d{2}/\d{2}/\d{2})"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna a data do contrato encontrada ou None
    dt_contrato = resultado.group(1) if resultado else None
    
    padrao = r"QTD CPA:\s*(\d+)"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna a quantidade de parcelas encontrada ou None
    qtd_cpa = resultado.group(1) if resultado else None
    
    
    padrao = r"(?:AG/CONTCGC|CPF|AG/CONTA-D/TIT|AG/CONTA-D/CPF)\s*:\s*([^\n\r]+)"
    
    # Procurar no texto pela ocorrência específica
    resultado = re.search(padrao, texto)
    
    # Retorna o CNPJ encontrado ou None
    cpf =  resultado.group(1) if resultado else None
    return [n_contrato, cliente, dt_contrato, qtd_cpa, cpf]


In [30]:
import glob



In [40]:
import pandas as pd

resultados = []
for arquivo in arquivos:
    try:
        with open(arquivo, 'r', encoding='utf-8') as f:
            texto = f.read()
    except UnicodeDecodeError:
        with open(arquivo, 'r', encoding='latin1') as f:
            texto = f.read()
    resp = f4_Tela_Consulta_de_bens(texto)
    resultados.append(resp)
    
arquivos = glob.glob("pdfs_2018/**/*_4 - Tela Consulta de bens L7RY V1.txt", recursive=True)
df_resultados = pd.DataFrame(resultados, columns=['n_contrato', 'dt_liquidacao', 'valor_bem'])
df_resultados.head()

,n_contrato,dt_liquidacao,valor_bem
0,0056533,08/11/18,15000.0
1,0069962,13/09/18,4000.0
2,0087851,13/09/18,28900.0
3,0131128,13/09/18,34472.0
4,0142133,13/09/18,27650.0


In [76]:
arquivos = glob.glob("pdfs_2018/**/*_1 - Tela Contrato L7RR v1.txt", recursive=True)
resultados = []
for arquivo in arquivos:
    try:
        with open(arquivo, 'r', encoding='utf-8') as f:
            texto = f.read()
    except UnicodeDecodeError:
        with open(arquivo, 'r', encoding='latin1') as f:
            texto = f.read()
    resp = Tela_Contrato_L7RR(texto)
    resultados.append(resp)

df_contratos = pd.DataFrame(resultados, columns=['n_contrato', 'cliente', 'dt_contrato', 'qtd_cpa', 'cpf'])
df_contratos.head()


,n_contrato,cliente,dt_contrato,qtd_cpa,cpf
0,8/0056533,JOCIMAR NERES MONTEIRO,13/07/06,0,095.037.887-90
1,1/0069962,HENRIQUE STAUT AYRES DE SOUZA,06/10/05,0,5432/01410-7/01
2,1/0087851,JOELSON TRINDADE DA SILVA,26/10/05,0,4054/02368-6/01
3,1/0131128,ROSIMEIRE ARCANJO DE OLIVEIRA,05/12/05,0,563.350.671-53
4,1/0142133,RUTH SOMBRA DE MENEZES,13/12/05,0,562.239.517-87


In [77]:
df_contratos["n_contrato"] = df_contratos["n_contrato"].str.split("/").str[-1]
df_contratos

,n_contrato,cliente,dt_contrato,qtd_cpa,cpf
0,0056533,JOCIMAR NERES MONTEIRO,13/07/06,0,095.037.887-90
1,0069962,HENRIQUE STAUT AYRES DE SOUZA,06/10/05,0,5432/01410-7/01
2,0087851,JOELSON TRINDADE DA SILVA,26/10/05,0,4054/02368-6/01
3,0131128,ROSIMEIRE ARCANJO DE OLIVEIRA,05/12/05,0,563.350.671-53
4,0142133,RUTH SOMBRA DE MENEZES,13/12/05,0,562.239.517-87
...,...,...,...,...,...
8695,5221451,MAPSOLO ENGENHARIA LTDA,28/08/14,0,0734/72114-5/01
8696,5221503,GV TRANS TRANSPORTES,04/09/14,0,6448/26500-6/01
8697,5222213,DUOFLY PARTICIPACOES LTDA,09/01/18,36,0445/61658-4/01
8698,5222215,POSTO TITAN CAFELANDIA LTDA,31/01/18,24,14.229.920/0001-90


In [78]:
df_merged = pd.merge(df_contratos, df_resultados, left_on="n_contrato", right_on="n_contrato", how="inner")
df_merged.head()

,n_contrato,cliente,dt_contrato,qtd_cpa,cpf,dt_liquidacao,valor_bem
0,0056533,JOCIMAR NERES MONTEIRO,13/07/06,0,095.037.887-90,08/11/18,15000.0
1,0069962,HENRIQUE STAUT AYRES DE SOUZA,06/10/05,0,5432/01410-7/01,13/09/18,4000.0
2,0087851,JOELSON TRINDADE DA SILVA,26/10/05,0,4054/02368-6/01,13/09/18,28900.0
3,0131128,ROSIMEIRE ARCANJO DE OLIVEIRA,05/12/05,0,563.350.671-53,13/09/18,34472.0
4,0142133,RUTH SOMBRA DE MENEZES,13/12/05,0,562.239.517-87,13/09/18,27650.0


In [ ]:
status_cols = ["Contrato","Cliente - Status","CNPJ/CPF - Status","Valor - Status","Data do Contrato - Status","Data de Liquidação - Status",'Quantidade de Parcelas - Status']


In [ ]:
df_base_completa = pd.read_excel(
    r"data\Input\DOC_BASE_2014.xlsx",
    # sheet_name="Base_2018",
    usecols="A:P",
    dtype={
        "n° do contrato": str
    }
)
df_base_completa.head()

In [57]:
df_base_completa = df_base_completa[["n° do contrato","razão social/nome completo do arrendatário","CNPJ/CPF do arrendatário","Valor do Bem","data do contrato","data de liquidação do contrato","n° de parcelas contratadas"]]
df_base_completa.columns =["n° do contrato","Razão Social_Nome Completo do Arrendatário","CNPJ_CPF do Arrendatário","Valor do Bem","data do contrato","Liquidação","N° de parcelas contratadas"]

df_base_completa.to_excel("base tabelas/DOC_BASE_Reprocessado (1).xlsx", sheet_name="Base_2018", index=False)


In [59]:
len(df_base_completa.columns)

7

In [61]:
from pandas import Timestamp


tipos_colunas = {
    'n° do contrato': str,   # Forçar coluna como string
    'Razão Social_Nome Completo do Arrendatário': str,   # Forçar coluna como string
    'CNPJ_CPF do Arrendatário': str,   # Forçar coluna como string
    'Nº de Parcelas Contratadas': str, # Forçar coluna como número
    # 'Prazo de vida útil normal do bem (em meses)': str, # Forçar coluna como número
    # 'Prazo de vida útil adotado para o bem (em meses)': str, # Forçar coluna como número
    'Valor do Bem': float, # Forçar coluna como número
    # 'Valor Total das Contraprestações de Arrendamento': float, # Forçar coluna como número
    # 'valor total do VRG': float
    # 'VRG ANTECIPADO': float, # Forçar coluna como número
    # 'VRG DILUIDO': float, # Forçar coluna como número
    # 'VRG FINAL': float, # Forçar coluna como número
}
data_columns = [
    'data do contrato', 
                # 'Data de Vencimento', 
                # 'Data Valor Atualizaçao ',
    # 'Data de Vencimento da primeira contraprestação',
    # 'Data de Vencimento da última contraprestação',
    # 'Data da Liquidação do Contrato',
    "Liquidação"
]
path_base = "base tabelas/DOC_BASE_Reprocessado (1).xlsx"
# 'Data Valor Atualizaçao , Data da Liquidação do Contrato, Data de Vencimento, Data do Contrato'
# Lendo o Excel, forçando os tipos de colunas e interpretando colunas de datas
df_base = pd.read_excel(path_base, dtype=tipos_colunas, parse_dates=data_columns)
df_base['N° de parcelas contratadas'] = pd.to_numeric(df_base['N° de parcelas contratadas'], errors='coerce').fillna(0).astype(int)
# df_base['Prazo de vida útil normal do bem (em meses);'] = pd.to_numeric(df_base['Prazo de vida útil normal do bem (em meses);'], errors='coerce').fillna(0).astype(int)
# df_base['Prazo de vida útil adotado para o bem (em meses);'] = pd.to_numeric(df_base['Prazo de vida útil adotado para o bem (em meses);'], errors='coerce').fillna(0).astype(int)

def formatar_cpf_cnpj(cpf_cnpj):
    if pd.isnull(cpf_cnpj):
        return cpf_cnpj
    cpf_cnpj = str(cpf_cnpj).strip()
    cpf_cnpj = ''.join(filter(str.isdigit, cpf_cnpj))  # Mantém apenas números

    if len(cpf_cnpj) == 14:  # CNPJ
        return f"{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}"
    elif len(cpf_cnpj) == 11:  # CPF
        return f"{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}"
    elif len(cpf_cnpj) < 11:  # CPF incompleto, preencher com zeros
        cpf_cnpj = cpf_cnpj.zfill(11)
        return f"{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}"
    elif len(cpf_cnpj) < 14:  # CNPJ incompleto, preencher com zeros
        cpf_cnpj = cpf_cnpj.zfill(14)
        return f"{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}"
    
    return cpf_cnpj  # Retorna o valor original se não for possível formatar
df_base['CNPJ_CPF do Arrendatário'] = df_base['CNPJ_CPF do Arrendatário'].apply(formatar_cpf_cnpj)
def to_pascal_case(s):
    if pd.isnull(s):
        return s
    return ' '.join(word.capitalize() for word in s.split(' '))
df_base['Razão Social_Nome Completo do Arrendatário'] = df_base['Razão Social_Nome Completo do Arrendatário'].apply(to_pascal_case)
df_base.groupby('n° do contrato').count().reset_index().sort_values('CNPJ_CPF do Arrendatário',ascending=False)

def format_date_columns(df, date_columns):
    # Iterar sobre as colunas de data e aplicar a conversão
    for col in date_columns:
        # Verifica se a coluna existe no dataframe
        if col in df.columns:
            # Converte as datas, forçando a coerção de erros
            df[col] = df[col].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, Timestamp) else pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.notnull(x) else '')
        else:
            print(f"Warning: Column '{col}' does not exist in the DataFrame.")
    return df


# Exemplo de uso
df_base = format_date_columns(df_base, data_columns)
df_base['n° do contrato'] = df_base['n° do contrato'].astype(str)  

C:\Users\patricklima\AppData\Local\Temp\ipykernel_24196\2160709565.py:67: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = df[col].apply(lambda x: x.strftime('%d/%m/%Y') if isinstance(x, Timestamp) else pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.notnull(x) else '')


In [67]:
df_base.columns

Index(['n° do contrato', 'Razão Social_Nome Completo do Arrendatário',
       'CNPJ_CPF do Arrendatário', 'Valor do Bem', 'data do contrato',
       'Liquidação', 'N° de parcelas contratadas'],
      dtype='object')

In [68]:
df_merged.columns


Index(['n_contrato', 'cliente', 'dt_contrato', 'qtd_cpa', 'cpf',
       'dt_liquidacao', 'valor_bem'],
      dtype='object')

In [82]:
def corrigir_ano(data):
    if pd.isnull(data):
        return data
    try:
        partes = data.split('/')
        if len(partes) == 3:
            dia, mes, ano = partes
            if len(ano) == 2:
                if int(ano) > 40:
                    ano_corrigido = '19' + ano
                else:
                    ano_corrigido = '20' + ano
                return f"{dia}/{mes}/{ano_corrigido}"
        return data
    except Exception:
        return data

df_merged['dt_contrato'] = df_merged['dt_contrato'].apply(corrigir_ano)
df_merged['dt_liquidacao'] = df_merged['dt_liquidacao'].apply(corrigir_ano)


In [93]:
"""
Para um mesmo 'n° do contrato' compare as colunas de df_merged e df_base
criando um novo dataframe df_merged_comparativo
coloque Conferido ou Divergente
compare as colunas :
    'n_contrato', 'cliente', 'cpf', 'valor_bem', 'dt_contrato', 'dt_liquidacao', 'qtd_cpa', 
       
    com as colunas:
    'n° do contrato', 'Razão Social_Nome Completo do Arrendatário',
       'CNPJ_CPF do Arrendatário', 'Valor do Bem', 'data do contrato',
       'Liquidação', 'N° de parcelas contratadas'
    
    status_cols = ["Contrato","Cliente - Status","CNPJ/CPF - Status","Valor - Status","Data do Contrato - Status","Data de Liquidação - Status",'Quantidade de Parcelas - Status']
"""
# Realiza o merge para alinhar os contratos
df_merged_comparativo = pd.merge(
    df_merged,
    df_base,
    left_on="n_contrato",
    right_on="n° do contrato",
    how="left",
    suffixes=('_extraido', '_base')
)

def compara_str(a, b):
    if pd.isnull(a) and pd.isnull(b):
        return "Conferido"
    if pd.isnull(a) or pd.isnull(b):
        return "Divergente"
    return "Conferido" if str(a).strip().upper() == str(b).strip().upper() else "Divergente"

def compara_num(a, b, tol=0.01):
    try:
        if pd.isnull(a) and pd.isnull(b):
            return "Conferido"
        if pd.isnull(a) or pd.isnull(b):
            return "Divergente"
        return "Conferido" if abs(float(a) - float(b)) < tol else "Divergente"
    except Exception:
        return "Divergente"

def compara_data(a, b):
    if pd.isnull(a) and pd.isnull(b):
        return "Conferido"
    if pd.isnull(a) or pd.isnull(b):
        return "Divergente"
    # Normaliza para dd/mm/aaaa
    try:
        a_fmt = pd.to_datetime(a, dayfirst=True, errors='coerce')
        b_fmt = pd.to_datetime(b, dayfirst=True, errors='coerce')
        if pd.isnull(a_fmt) or pd.isnull(b_fmt):
            return "Divergente"
        return "Conferido" if a_fmt == b_fmt else "Divergente"
    except Exception:
        return "Divergente"

df_merged_comparativo["Contrato"] = df_merged_comparativo.apply(
    lambda x: compara_str(x["n_contrato"], x["n° do contrato"]), axis=1)
df_merged_comparativo["Cliente - Status"] = df_merged_comparativo.apply(
    lambda x: compara_str(x["cliente"], x["Razão Social_Nome Completo do Arrendatário"]), axis=1)
df_merged_comparativo["CNPJ/CPF - Status"] = df_merged_comparativo.apply(
    lambda x: compara_str(
        ''.join(filter(str.isdigit, str(x["cpf"]))),
        ''.join(filter(str.isdigit, str(x["CNPJ_CPF do Arrendatário"])))
    ), axis=1)
df_merged_comparativo["Valor - Status"] = df_merged_comparativo.apply(
    lambda x: compara_num(x["valor_bem"], x["Valor do Bem"]), axis=1)
df_merged_comparativo["Data do Contrato - Status"] = df_merged_comparativo.apply(
    lambda x: compara_data(x["dt_contrato"], x["data do contrato"]), axis=1)
df_merged_comparativo["Data de Liquidação - Status"] = df_merged_comparativo.apply(
    lambda x: compara_data(x["dt_liquidacao"], x["Liquidação"]), axis=1)
df_merged_comparativo["Quantidade de Parcelas - Status"] = df_merged_comparativo.apply(
    lambda x: compara_num(x["qtd_cpa"], x["N° de parcelas contratadas"]), axis=1)
# df_merged_comparativo.groupby("CNPJ/CPF - Status")[["Contrato"]].count()
# df_merged_comparativo.groupby("Cliente - Status")[["Contrato"]].count()
df_merged_comparativo["CNPJ/CPF - Status"] = df_merged_comparativo.apply(lambda x: "Conferido" if x["Cliente - Status"] == "Conferido" else x["CNPJ/CPF - Status"], axis=1)
df_merged_comparativo["Cliente - Status"] = df_merged_comparativo.apply(lambda x: "Conferido" if x["CNPJ/CPF - Status"] == "Conferido" else x["Cliente - Status"], axis=1)

df_merged_comparativo.head()


,n_contrato,cliente,dt_contrato,qtd_cpa,cpf,dt_liquidacao,valor_bem,n° do contrato,Razão Social_Nome Completo do Arrendatário,CNPJ_CPF do Arrendatário,...,data do contrato,Liquidação,N° de parcelas contratadas,Contrato,Cliente - Status,CNPJ/CPF - Status,Valor - Status,Data do Contrato - Status,Data de Liquidação - Status,Quantidade de Parcelas - Status
0,0056533,JOCIMAR NERES MONTEIRO,13/07/2006,0,095.037.887-90,08/11/2018,15000.0,0056533,Jocimar Neres Monteiro,00.009.503/7887-90,...,13/07/2006,08/11/2018,48.0,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
1,0069962,HENRIQUE STAUT AYRES DE SOUZA,06/10/2005,0,5432/01410-7/01,13/09/2018,4000.0,0069962,Henrique Staut Ayres De Souza,00.027.928/7368-02,...,06/10/2005,13/09/2018,24.0,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
2,0087851,JOELSON TRINDADE DA SILVA,26/10/2005,0,4054/02368-6/01,13/09/2018,28900.0,0087851,Joelson Trindade Da Silva,00.056.958/8032-72,...,26/10/2005,13/09/2018,36.0,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
3,0131128,ROSIMEIRE ARCANJO DE OLIVEIRA,05/12/2005,0,563.350.671-53,13/09/2018,34472.0,0131128,Rosimeire Arcanjo De Oliveira,00.056.335/0671-53,...,05/12/2005,13/09/2018,60.0,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
4,0142133,RUTH SOMBRA DE MENEZES,13/12/2005,0,562.239.517-87,13/09/2018,27650.0,0142133,Ruth Sombra De Menezes,00.056.223/9517-87,...,13/12/2005,13/09/2018,60.0,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente


In [94]:
status_cols = ["Contrato","Cliente - Status","CNPJ/CPF - Status","Valor - Status","Data do Contrato - Status","Data de Liquidação - Status",'Quantidade de Parcelas - Status']
df_merged_comparativo["Contrato"] = df_merged_comparativo["n_contrato"]
df_merged_comparativo[status_cols]

,Contrato,Cliente - Status,CNPJ/CPF - Status,Valor - Status,Data do Contrato - Status,Data de Liquidação - Status,Quantidade de Parcelas - Status
0,0056533,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
1,0069962,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
2,0087851,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
3,0131128,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
4,0142133,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
...,...,...,...,...,...,...,...
8695,5221451,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
8696,5221503,Conferido,Conferido,Conferido,Conferido,Conferido,Divergente
8697,5222213,Conferido,Conferido,Conferido,Conferido,Conferido,Conferido
8698,5222215,Divergente,Divergente,Divergente,Divergente,Divergente,Divergente


In [101]:
df_merged_comparativo.groupby("Cliente - Status")[["Contrato"]].count()

,Contrato
Cliente - Status,
Conferido,8677
Divergente,23


In [102]:
df_merged_comparativo.groupby("CNPJ/CPF - Status")[["Contrato"]].count()

,Contrato
CNPJ/CPF - Status,
Conferido,8677
Divergente,23


In [103]:
df_merged_comparativo.groupby("Valor - Status")[["Contrato"]].count()

,Contrato
Valor - Status,
Conferido,8690
Divergente,10


In [104]:
df_merged_comparativo.groupby("Data do Contrato - Status")[["Contrato"]].count()

,Contrato
Data do Contrato - Status,
Conferido,8690
Divergente,10


In [105]:
df_merged_comparativo.groupby("Data de Liquidação - Status")[["Contrato"]].count()

,Contrato
Data de Liquidação - Status,
Conferido,8650
Divergente,50


In [106]:

df_merged_comparativo.groupby("Quantidade de Parcelas - Status")[["Contrato"]].count()

,Contrato
Quantidade de Parcelas - Status,
Conferido,1122
Divergente,7578


In [108]:
df_merged_comparativo[status_cols].to_excel("base tabelas/Status_2018.xlsx", index=False)

In [115]:
df_contratos_sem_zeros = pd.read_excel("2018 faltando 2.xlsx")
df_contratos_sem_zeros.head()

,Contrato
0,56533
1,69962
2,87851
3,131128
4,142133


In [116]:
df_contratos_sem_zeros["Contrato"] = df_contratos_sem_zeros["Contrato"].astype(str).str.zfill(7)
df_contratos_sem_zeros

,Contrato
0,0056533
1,0069962
2,0087851
3,0131128
4,0142133
...,...
8685,5076803
8686,5221451
8687,5221503
8688,5222213


In [117]:
import os
import shutil

# Caminho base das pastas
pasta_entrega = "Entrega"
pasta_invalidos = "invalidos"

# Garante que a pasta de inválidos existe
os.makedirs(pasta_invalidos, exist_ok=True)

# Lista de contratos como strings (sem zeros à esquerda)
contratos_validos = set(df_contratos_sem_zeros["Contrato"].astype(str))

# Percorre todos os arquivos nas subpastas de Entrega
for root, dirs, files in os.walk(pasta_entrega):
    for nome_arquivo in files:
        caminho_arquivo = os.path.join(root, nome_arquivo)
        # Verifica se algum contrato está no nome do arquivo
        if not any(str(contrato) in nome_arquivo for contrato in contratos_validos):
            # Move para a pasta invalidos (mantendo apenas o nome do arquivo)
            destino = os.path.join(pasta_invalidos, nome_arquivo)
            shutil.move(caminho_arquivo, destino)

In [120]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Caminho do arquivo de contratos a filtrar
arquivo_filtrar = "filtrar.csv"  # ajuste para o nome/caminho correto do arquivo

# Lê a coluna 'Contrato' do arquivo de filtro
df_filtrar = pd.read_csv(arquivo_filtrar, sep=";")  # ajuste o sep se necessário

# Cria a pasta entrega_parcial se não existir
os.makedirs("entrega_parcial", exist_ok=True)
contratos_filtrar = set(df_filtrar["Contrato"].astype(str))

# Pastas de origem e destino
pasta_entrega = "Entrega"
pasta_destino = "entrega_parcial"

for root, dirs, files in os.walk(pasta_entrega):
    for nome_arquivo in files:
        # Verifica se algum contrato está no nome do arquivo
        if any(str(contrato) in nome_arquivo for contrato in contratos_filtrar):
            caminho_origem = os.path.join(root, nome_arquivo)
            # Calcula o caminho relativo para manter a estrutura de pastas
            caminho_relativo = os.path.relpath(root, pasta_entrega)
            destino_dir = os.path.join(pasta_destino, caminho_relativo)
            os.makedirs(destino_dir, exist_ok=True)
            caminho_destino = os.path.join(destino_dir, nome_arquivo)
            shutil.copy2(caminho_origem, caminho_destino)